In [3]:
import cv2
import numpy as np

def cartoonize(image):
    # 이미지를 회색조로 변환
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # 이미지를 부드럽게 만들기 위해 가우시안 블러 적용
    gray = cv2.medianBlur(gray, 5)
    
    # 엣지 감지
    edges = cv2.Laplacian(gray, cv2.CV_8U, ksize=5)
    
    # 이진화
    ret, mask = cv2.threshold(edges, 100, 255, cv2.THRESH_BINARY_INV)
    
    # 색상 전환
    dst = cv2.bilateralFilter(image, d=9, sigmaColor=75, sigmaSpace=75)
    
    # 결과 이미지 생성
    cartoon = cv2.bitwise_and(dst, dst, mask=mask)
    
    return cartoon

# 영상 파일 경로
input_file = 'input_video.mp4'
output_file = 'output_video_cartoon.mp4'

# 비디오 불러오기
cap = cv2.VideoCapture(input_file)
fps = cap.get(cv2.CAP_PROP_FPS)/2
frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

# 비디오 작성자 객체 생성
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter(output_file, fourcc, fps, frame_size)

# 비디오 프레임별로 처리
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 프레임을 카툰 스타일로 변환
    cartoon_frame = cartoonize(frame)
    
    # 결과 비디오에 프레임 쓰기
    out.write(cartoon_frame)
    
    # 화면에 결과 출력
    cv2.imshow('Cartoonized Video', np.hstack((frame[:,:640,:], cartoon_frame[:,640:,:])))
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 종료
cap.release()
out.release()
cv2.destroyAllWindows()